<a href="https://colab.research.google.com/github/MicaelaRomeroC3/COPs/blob/main/06_Tablacategorias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import os
import numpy as np

In [3]:
ruta_historical ='/content/drive/MyDrive/COPS/DATA/3.HISTORICAL/llamadas.parquet'
ruta_salida = '/content/drive/MyDrive/COPS/OUTPUT/TABLA_CATEGORIAS.xlsx'
os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)

# === Cargar histórico ===
if os.path.exists(ruta_historical):
    df = pd.read_parquet(ruta_historical)
    print(f"📊 Histórico cargado con {len(df)} registros.")
else:
    print("⚠️ No se encontró el archivo histórico.")
    exit()

📊 Histórico cargado con 9992 registros.


In [4]:
# ANTIGUO

# === Variables de cada tipo ===
columnas_eval_1 = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'S1', 'S2', 'S3', 'S4', 'S5M', 'C1M', 'C2M', 'T1M', 'T2M', 'T3']
ponderacion_dict = {
    'P1': 0.02, 'P2': 0.10, 'P3': 0.03, 'P4': 0.03, 'P5': 0.05, 'P6': 0.02,
    'S1': 0.03, 'S2': 0.02, 'S3': 0.10, 'S4': 0.10, 'S5M': 0.05,
    'C1M': 0.10, 'C2M': 0.10, 'T1M': 0.05, 'T2M': 0.10, 'T3': 0.10
}

# === Lista de dimensiones ===
dimensiones = ['PRODUCTO', 'SEGMENTO', 'R_NR', 'PERIMETRO']
tablas_por_dimension = []

# === Función para generar análisis por dimensión ===
for dim in dimensiones:
    errores = df.groupby(['FECHA_EVALUACIÓN', dim])[columnas_eval_1].apply(lambda x: (x == False).sum()).reset_index()
    llamadas = df.groupby(['FECHA_EVALUACIÓN', dim]).size().reset_index(name='NUM_LLAMADAS')
    resultado = pd.merge(errores, llamadas, on=['FECHA_EVALUACIÓN', dim])

    resultado = resultado.melt(
        id_vars=['FECHA_EVALUACIÓN', dim, 'NUM_LLAMADAS'],
        value_vars=columnas_eval_1,
        var_name='VARIABLE',
        value_name='N_FALSOS'
    )

    resultado['%_CUMPLIMIENTO'] = ((1 - resultado['N_FALSOS'] / resultado['NUM_LLAMADAS']) * 100).round(2)
    resultado['PONDERACION'] = resultado['VARIABLE'].map(ponderacion_dict)

    resultado['DIMENSIÓN'] = dim.capitalize()
    resultado['VALOR_DIMENSIÓN'] = resultado[dim]
    resultado.drop(columns=[dim], inplace=True)

    tablas_por_dimension.append(resultado)

# === Concatenar todas las tablas por dimensión en una tabla general ===
tabla_general = pd.concat(tablas_por_dimension, ignore_index=True)

# === Cálculo %_CUMPLIMIENTO_P, C, S, T ===
def calcular_cumplimiento_grupo(df, grupo, divisor):
    variables_grupo = {
        'P': ['P1', 'P2', 'P3', 'P4', 'P5', 'P6'],
        'S': ['S1', 'S2', 'S3', 'S4', 'S5M'],
        'C': ['C1M', 'C2M'],
        'T': ['T1M', 'T2M', 'T3']
    }
    temp = df[df['VARIABLE'].isin(variables_grupo[grupo])].copy()
    temp['PON_CUMP'] = temp['%_CUMPLIMIENTO'] * temp['PONDERACION']
    resultado = (
        temp.groupby(['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'])['PON_CUMP'].sum().reset_index()
    )
    resultado[f'%_CUMPLIMIENTO_{grupo}'] = (resultado['PON_CUMP'] / divisor).round(2)
    resultado.drop(columns='PON_CUMP', inplace=True)
    return resultado

cumplimiento_P = calcular_cumplimiento_grupo(tabla_general, 'P', 0.25)
cumplimiento_S = calcular_cumplimiento_grupo(tabla_general, 'S', 0.30)
cumplimiento_C = calcular_cumplimiento_grupo(tabla_general, 'C', 0.20)
cumplimiento_T = calcular_cumplimiento_grupo(tabla_general, 'T', 0.25)

# === Añadir a tabla_general ===
for df_cumpl in [cumplimiento_P, cumplimiento_S, cumplimiento_C, cumplimiento_T]:
    tabla_general = pd.merge(tabla_general, df_cumpl, on=['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'], how='left')

# === Añadir %_CUMPLIMIENTO_G (global) ===
def calcular_cumplimiento_global(df):
    temp = df[df['VARIABLE'].isin(columnas_eval_1)].copy()
    temp['PON_CUMP'] = temp['PONDERACION'] * temp['%_CUMPLIMIENTO']
    resultado = (
        temp.groupby(['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'])['PON_CUMP'].sum()
        .reset_index()
        .rename(columns={'PON_CUMP': '%_CUMPLIMIENTO_G'})
    )
    return resultado

cumplimiento_global = calcular_cumplimiento_global(tabla_general)
tabla_general = pd.merge(tabla_general, cumplimiento_global, on=['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'], how='left')


In [5]:
dimensiones_resumen = ['PRODUCTO', 'SEGMENTO', 'R_NR', 'PERIMETRO']
resumen_globales = {}

for dim in dimensiones_resumen:
    # 1️⃣ Calcular cantidad de errores (falsos)
    errores = df.groupby(dim)[columnas_eval_1].apply(lambda x: (x == False).sum()).reset_index()

    # 2️⃣ Calcular cantidad de llamadas (muestra total)
    llamadas = df.groupby(dim).size().reset_index(name='NUM_LLAMADAS')

    # 3️⃣ Unir errores con cantidad de llamadas
    temp = pd.merge(errores, llamadas, on=dim)

    # 4️⃣ Pasar a formato largo
    temp = temp.melt(
        id_vars=[dim, 'NUM_LLAMADAS'],
        value_vars=columnas_eval_1,
        var_name='VARIABLE',
        value_name='N_FALSOS'
    )

    # 5️⃣ Calcular % cumplimiento y ponderaciones
    temp['%_CUMPLIMIENTO'] = ((1 - temp['N_FALSOS'] / temp['NUM_LLAMADAS']) * 100).round(2)
    temp['PONDERACION'] = temp['VARIABLE'].map(ponderacion_dict)
    temp['PON_CUMP'] = (temp['%_CUMPLIMIENTO'] / 100) * temp['PONDERACION']

    # 6️⃣ Agregar resultado ponderado por dimensión
    resumen = temp.groupby(dim, as_index=False).agg({
        'PON_CUMP': 'sum',
        'NUM_LLAMADAS': 'first'  # o 'sum', si el merge generó duplicados
    })

    resumen['%_CUMPLIMIENTO_GLOBAL'] = (resumen['PON_CUMP'] * 100).round(2)

    # 7️⃣ Guardar en el diccionario incluyendo la cantidad de registros
    resumen_globales[dim] = resumen[[dim, 'NUM_LLAMADAS', '%_CUMPLIMIENTO_GLOBAL']]

# Ejemplo: ver resultado de la última dimensión
print(resumen_globales[dim])


  PERIMETRO  NUM_LLAMADAS  %_CUMPLIMIENTO_GLOBAL
0      0-30          3150                  99.47
1   180-365          2270                  99.06
2     30-90          1846                  99.66
3      365m           800                  99.72
4    90-180          1926                  99.33


In [6]:
# === Variables y ponderaciones ===
columnas_eval_1 = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6',
                   'S1', 'S2', 'S3', 'S4', 'S5M',
                   'C1M', 'C2M',
                   'T1M', 'T2M', 'T3']

ponderacion_dict = {
    'P1': 0.02, 'P2': 0.10, 'P3': 0.03, 'P4': 0.03, 'P5': 0.05, 'P6': 0.02,
    'S1': 0.03, 'S2': 0.02, 'S3': 0.10, 'S4': 0.10, 'S5M': 0.05,
    'C1M': 0.10, 'C2M': 0.10,
    'T1M': 0.05, 'T2M': 0.10, 'T3': 0.10
}

# === Asegurar que FECHA_EVALUACIÓN sea datetime ===
df['FECHA_EVALUACIÓN'] = pd.to_datetime(df['FECHA_EVALUACIÓN'], dayfirst=True)

# === Función para calcular primer día hábil después de la evaluación ===
def siguiente_dia_habil(fecha):
    dia = fecha + pd.Timedelta(days=1)
    while dia.weekday() >= 5:  # 5=sábado, 6=domingo
        dia += pd.Timedelta(days=1)
    return dia

df['FECHA_INFORME'] = df['FECHA_EVALUACIÓN'].apply(siguiente_dia_habil)

# === Preparar tabla para cálculo ===
tabla = df.copy()
tabla['NUM_LLAMADAS'] = 1  # cada fila representa una llamada

tabla_melt = tabla.melt(
    id_vars=['FECHA_EVALUACIÓN', 'FECHA_INFORME', 'SEGMENTO'],
    value_vars=columnas_eval_1,
    var_name='VARIABLE',
    value_name='ACIERTO'
)

# Transformar a 0/1: True = 1, False = 0
tabla_melt['ACIERTO'] = tabla_melt['ACIERTO'].astype(int)
tabla_melt['PONDERACION'] = tabla_melt['VARIABLE'].map(ponderacion_dict)
tabla_melt['PON_CUMP'] = tabla_melt['ACIERTO'] * tabla_melt['PONDERACION']

# === Filtrar por SEGMENTO 'mas90' y 'menos90' ===
tabla_filtrada = tabla_melt[tabla_melt['SEGMENTO'].isin(['mas90', 'menos90'])].copy()

# === % de cumplimiento ponderado por FECHA_EVALUACIÓN, FECHA_INFORME y SEGMENTO ===
cumplimiento_fecha_dim = (
    tabla_filtrada
    .groupby(['FECHA_EVALUACIÓN', 'FECHA_INFORME', 'SEGMENTO'], as_index=False)
    .agg({'PON_CUMP':'sum', 'PONDERACION':'sum'})
)
cumplimiento_fecha_dim['%_CUMPLIMIENTO_GLOBAL'] = ((cumplimiento_fecha_dim['PON_CUMP'] / cumplimiento_fecha_dim['PONDERACION']) * 100).round(2)
cumplimiento_fecha_dim.rename(columns={'SEGMENTO':'VALOR_DIMENSIÓN'}, inplace=True)

# === Calcular GLOBAL por FECHA_EVALUACIÓN y FECHA_INFORME ===
cumplimiento_global = (
    cumplimiento_fecha_dim
    .groupby(['FECHA_EVALUACIÓN', 'FECHA_INFORME'], as_index=False)
    .agg({'PON_CUMP':'sum', 'PONDERACION':'sum'})
)
cumplimiento_global['%_CUMPLIMIENTO_GLOBAL'] = ((cumplimiento_global['PON_CUMP'] / cumplimiento_global['PONDERACION']) * 100).round(2)
cumplimiento_global['VALOR_DIMENSIÓN'] = 'GLOBAL'

# === Unir ambos resultados ===
tabla_cumplimiento_final = pd.concat([cumplimiento_fecha_dim, cumplimiento_global], ignore_index=True)

# === Ordenar y limpiar columnas finales ===
tabla_cumplimiento_final = tabla_cumplimiento_final.sort_values(['FECHA_EVALUACIÓN', 'FECHA_INFORME', 'VALOR_DIMENSIÓN'])
tabla_cumplimiento_final = tabla_cumplimiento_final[['FECHA_EVALUACIÓN', 'FECHA_INFORME', 'VALOR_DIMENSIÓN', '%_CUMPLIMIENTO_GLOBAL']]

tabla_cumplimiento_final.head(20)


,FECHA_EVALUACIÓN,FECHA_INFORME,VALOR_DIMENSIÓN,%_CUMPLIMIENTO_GLOBAL
44,2025-04-01,2025-04-02,GLOBAL,99.44
0,2025-04-01,2025-04-02,mas90,99.80
1,2025-04-01,2025-04-02,menos90,99.06
45,2025-04-02,2025-04-03,GLOBAL,99.42
2,2025-04-02,2025-04-03,mas90,99.76
3,2025-04-02,2025-04-03,menos90,99.07
46,2025-04-03,2025-04-04,GLOBAL,99.62
4,2025-04-03,2025-04-04,mas90,99.86
5,2025-04-03,2025-04-04,menos90,99.41
47,2025-04-04,2025-04-07,GLOBAL,99.27


In [7]:
# === Agregar columna que identifica el tipo de variable (la primera letra) ===
tabla_melt['TIPO_VARIABLE'] = tabla_melt['VARIABLE'].str[0]

# === Calcular cumplimiento total por tipo (P, S, C, T) ===
cumplimiento_tipo_total = (
    tabla_melt
    .groupby('TIPO_VARIABLE', as_index=False)
    .agg({'PON_CUMP': 'sum', 'PONDERACION': 'sum'})
)
cumplimiento_tipo_total['%_CUMPLIMIENTO_GLOBAL'] = (
    (cumplimiento_tipo_total['PON_CUMP'] / cumplimiento_tipo_total['PONDERACION']) * 100
).round(2)

# === Calcular cumplimiento individual por variable ===
cumplimiento_variable_total = (
    tabla_melt
    .groupby(['TIPO_VARIABLE', 'VARIABLE'], as_index=False)
    .agg({'PON_CUMP': 'sum', 'PONDERACION': 'sum'})
)
cumplimiento_variable_total['%_CUMPLIMIENTO_GLOBAL'] = (
    (cumplimiento_variable_total['PON_CUMP'] / cumplimiento_variable_total['PONDERACION']) * 100
).round(2)

# === Agregar columna de descripción de la variable ===
mapeo_descripciones = {
    'P1': 'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA',
    'P2': 'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD',
    'P3': 'ESCUCHA COMPRENSIVA Y EMPÁTICA',
    'P4': 'TRATO CORDIAL AL CLIENTE',
    'P6': 'RESPONSABILIDAD CORPORATIVA',
    'P5': 'DESPEDIDA CORPORATIVA',
    'S1': 'INFORMACIÓN MOTIVO DE LA LLAMADA',
    'S2': 'SITUACIÓN ACTUAL DEL DEUDOR',
    'S3': 'SONDEO: IDENTIFICAR CAPACIDAD DE PAGO Y DETECCIÓN DE NECESIDADES',
    'S4': 'DIRECCIÓN',
    'S5M': 'PREPARACIÓN CIERRE : RESUMEN DE ACUERDOS',
    'C1M': 'OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO',
    'C2M': 'PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS PARA REALIZAR LOS INGRESOS LO ANTES POSIBLE',
    'T1M': 'USO EFECTIVO DEL SCRIPT',
    'T2M': 'CAPACIDAD DE EXPRESIÓN',
    'T3': 'BUSCAR EL COMPROMISO DE PAGO ENUNCIANDO LOS BENEFICIOS Y LAS CONSECUENCIAS DE LA REGULARIZACIÓN'
}

cumplimiento_variable_total['DESCRIPCION_VARIABLE'] = cumplimiento_variable_total['VARIABLE'].map(mapeo_descripciones)

# Las filas TOTAL estarán vacías en descripción
cumplimiento_tipo_total['VARIABLE'] = 'TOTAL'
cumplimiento_tipo_total['DESCRIPCION_VARIABLE'] = ''

# === Combinar totales por tipo y variables individuales ===
tabla_categorias = pd.concat([
    cumplimiento_tipo_total[['TIPO_VARIABLE', 'VARIABLE', 'DESCRIPCION_VARIABLE', '%_CUMPLIMIENTO_GLOBAL']],
    cumplimiento_variable_total[['TIPO_VARIABLE', 'VARIABLE', 'DESCRIPCION_VARIABLE', '%_CUMPLIMIENTO_GLOBAL']]
], ignore_index=True)

# === Ordenar por tipo y variable ===
tabla_categorias = tabla_categorias.sort_values(['TIPO_VARIABLE', 'VARIABLE']).reset_index(drop=True)

# Mostrar resultado final
print(tabla_categorias)



   TIPO_VARIABLE VARIABLE                               DESCRIPCION_VARIABLE  \
0              C      C1M   OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO   
1              C      C2M  PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS ...   
2              C    TOTAL                                                      
3              P       P1  PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN D...   
4              P       P2             IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD   
5              P       P3                     ESCUCHA COMPRENSIVA Y EMPÁTICA   
6              P       P4                           TRATO CORDIAL AL CLIENTE   
7              P       P5                              DESPEDIDA CORPORATIVA   
8              P       P6                        RESPONSABILIDAD CORPORATIVA   
9              P    TOTAL                                                      
10             S       S1                   INFORMACIÓN MOTIVO DE LA LLAMADA   
11             S       S2               

In [15]:
# === Derivar EMPRESA a partir de SERVICIO ===
if 'SERVICIO' not in df.columns:
    raise ValueError("⚠️ La columna 'SERVICIO' no existe en el DataFrame original.")

# Crear columna EMPRESA tomando lo que está antes del '_'
df['EMPRESA'] = df['SERVICIO'].str.split('_').str[0]

# Asegurar que FECHA_EVALUACIÓN es datetime
df['FECHA_EVALUACIÓN'] = pd.to_datetime(df['FECHA_EVALUACIÓN'], dayfirst=True, errors='coerce')

# === Crear columna MES en español sin usar locale ===
meses_es = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}
df['MES'] = df['FECHA_EVALUACIÓN'].dt.month.map(meses_es)

# === Recalcular tabla base con EMPRESA, SEGMENTO y MES ===
tabla_emp = df.melt(
    id_vars=['EMPRESA', 'SEGMENTO', 'MES'],
    value_vars=columnas_eval_1,
    var_name='VARIABLE',
    value_name='ACIERTO'
)

# Transformar a 0/1
tabla_emp['ACIERTO'] = tabla_emp['ACIERTO'].astype(int)
tabla_emp['PONDERACION'] = tabla_emp['VARIABLE'].map(ponderacion_dict)
tabla_emp['PON_CUMP'] = tabla_emp['ACIERTO'] * tabla_emp['PONDERACION']

# === Calcular % cumplimiento ponderado por EMPRESA, SEGMENTO y MES ===
cumplimiento_empresa_seg = (
    tabla_emp
    .groupby(['EMPRESA', 'SEGMENTO', 'MES'], as_index=False)
    .agg({'PON_CUMP': 'sum', 'PONDERACION': 'sum'})
)
cumplimiento_empresa_seg['%_CUMPLIMIENTO'] = (
    (cumplimiento_empresa_seg['PON_CUMP'] / cumplimiento_empresa_seg['PONDERACION']) * 100
).round(2)

# === Pivotear para tener columnas separadas por segmento ===
tabla_empresa = cumplimiento_empresa_seg.pivot_table(
    index=['EMPRESA', 'MES'],
    columns='SEGMENTO',
    values='%_CUMPLIMIENTO'
).reset_index()

# === Calcular el cumplimiento GLOBAL (ponderado total) ===
cumplimiento_empresa_global = (
    tabla_emp
    .groupby(['EMPRESA', 'MES'], as_index=False)
    .agg({'PON_CUMP': 'sum', 'PONDERACION': 'sum'})
)
cumplimiento_empresa_global['%_CUMPLIMIENTO_GLOBAL'] = (
    (cumplimiento_empresa_global['PON_CUMP'] / cumplimiento_empresa_global['PONDERACION']) * 100
).round(2)

# === Unir todo en una tabla final ===
tabla_cumplimiento_empresa = pd.merge(
    cumplimiento_empresa_global[['EMPRESA', 'MES', '%_CUMPLIMIENTO_GLOBAL']],
    tabla_empresa,
    on=['EMPRESA', 'MES'],
    how='left'
)

# === Renombrar columnas ===
tabla_cumplimiento_empresa.rename(columns={
    'menos90': '%_CUMPLIMIENTO_0_90',
    'mas90': '%_CUMPLIMIENTO_MAS_90'
}, inplace=True)

# === (Opcional) Calcular fila GLOBAL ponderada de todas las empresas ===
total = cumplimiento_empresa_global[['PON_CUMP', 'PONDERACION']].sum()
fila_global = pd.DataFrame({
    'EMPRESA': ['GLOBAL'],
    'MES': ['TOTAL'],
    '%_CUMPLIMIENTO_GLOBAL': [round((total['PON_CUMP'] / total['PONDERACION']) * 100, 2)],
    '%_CUMPLIMIENTO_0_90': [tabla_cumplimiento_empresa['%_CUMPLIMIENTO_0_90'].mean()],
    '%_CUMPLIMIENTO_MAS_90': [tabla_cumplimiento_empresa['%_CUMPLIMIENTO_MAS_90'].mean()]
})

tabla_cumplimiento_empresa = pd.concat([tabla_cumplimiento_empresa, fila_global], ignore_index=True)

# === Mostrar resultado final ===
print("✅ Cumplimiento por EMPRESA y MES:")
print(tabla_cumplimiento_empresa)



✅ Cumplimiento por EMPRESA y MES:
  EMPRESA    MES  %_CUMPLIMIENTO_GLOBAL  %_CUMPLIMIENTO_MAS_90  \
0     ADV  Abril                 100.00                100.000   
1     AXA  Abril                  99.31                 98.910   
2     COL  Abril                  99.33                100.000   
3     DXC  Abril                  99.48                 99.490   
4     ESC  Abril                  99.74                 99.620   
5    GCBE  Abril                  99.25                 99.250   
6     INT  Abril                  99.75                 99.750   
7     LEX  Abril                  98.54                 98.540   
8  GLOBAL  TOTAL                  99.40                 99.445   

   %_CUMPLIMIENTO_0_90  
0                  NaN  
1              99.4500  
2              99.3300  
3              98.8900  
4              99.8000  
5                  NaN  
6                  NaN  
7                  NaN  
8              99.3675  


In [17]:
# === Exportar a Excel junto con otras hojas ===
with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='w') as writer:

    # 1️⃣ Hojas detalladas por dimensión
    for dim in dimensiones:
        df_dim = tabla_general[tabla_general['DIMENSIÓN'] == dim.capitalize()]
        df_dim.to_excel(writer, sheet_name=dim.capitalize(), index=False)

    # 2️⃣ Tabla general concatenada
    tabla_general.to_excel(writer, sheet_name='TABLA_GENERALCATEG', index=False)

    # 3️⃣ Resumen global por dimensión
    for dim, df_resumen in resumen_globales.items():
        df_resumen.to_excel(writer, sheet_name=f'{dim.capitalize()}_GLOBAL', index=False)

    # 4️⃣ Cumplimiento mas90 / menos90 + global por fecha
    tabla_cumplimiento_final.to_excel(writer, sheet_name='CUMPLIMIENTO_90', index=False)

    # 5️⃣ Tabla categorías con detalle y totales por tipo
    tabla_categorias.to_excel(writer, sheet_name='TABLA_CATEGORIAS', index=False)

    # ➕ 6️⃣ Nueva hoja con cumplimiento por EMPRESA
    tabla_cumplimiento_empresa.to_excel(writer, sheet_name='CUMPLIMIENTO_EMPRESA', index=False)

print(f"✅ Excel generado correctamente en: {ruta_salida}")


✅ Excel generado correctamente en: /content/drive/MyDrive/COPS/OUTPUT/TABLA_CATEGORIAS.xlsx
